## Projekt Python: Wykrywanie spamu w SMS

**Zaimportowanie niezbędnych bibliotek i narzędzi**

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olejk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import i przygotowanie danych

In [22]:

sms = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t', names=["label", "message"])
sms = sms[['message', 'label']]
all_sms = []

**Pętla z każdego smsa pozbywa sie znaków specjalnych (np. : ; , . # $ ^), słów o małym znaczeniu (stop words)
oraz pozbywa się końcówek fleksyjnych ze słów i pozostawia temat wyrazu (stemming, algorytm Portera)**

In [23]:
for i in range(0, 5572):
    message = re.sub('[^a-zA-Z]', ' ', sms['message'][i] )
    message = message.lower()
    message = message.split()
    ps = PorterStemmer()
    message = [ps.stem(word) for word in message if not word in set(stopwords.words('english'))]
    message = ' '.join(message)
    all_sms.append(message)



**Stworzenie Bag of Words (vectorisation)**

In [24]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 6000)
X = cv.fit_transform(all_sms).toarray()
y = sms.iloc[:, 1]
             



**Rozdzielenie zbioru na dane treningowe i testowe**

In [26]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

**Zastosowanie klasyfikatora Naive Bayes do danych treningowych**

In [27]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None)

**Zastosowanie klasyfikatora Naive Bayes do danych testowych**

In [28]:
y_pred = classifier.predict(X_test)

**Macierz pomyłek**

In [29]:
from sklearn.metrics import confusion_matrix    
cm = confusion_matrix(y_test, y_pred)

In [30]:
cm

array([[1041,  167],
       [  19,  166]])